In [71]:
import datetime as dt
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn import preprocessing

# Data Input, EDA & Preprocessing

## Individual Data Input

In [72]:
price_vol = pd.read_csv('../data/Price & Volume BTC.csv')
tweets = pd.read_csv('../data/Tweets.csv')
funding_rates_1600 = pd.read_csv('../data/Funding Rates BTCUSDT 1600.csv')
funding_rates_0800 = pd.read_csv('../data/Funding Rates BTCUSDT 0800.csv')
funding_rates_0000 = pd.read_csv('../data/Funding Rates BTCUSDT 0000.csv')
google = pd.read_csv('../data/Google.csv')
transactions = pd.read_csv('../data/Transactions.csv')
unique_addresses = pd.read_csv('../data/Unique Addresses.csv')
active_addresses = pd.read_csv('../data/Active Addresses.csv')

## Data Compilation

In [73]:
# get daily ave value for twitter attributes
tweets = tweets.groupby(by="Time").mean()
tweets = tweets.reset_index()

df = pd.merge(price_vol, funding_rates_0000, on=['Time'])
df = pd.merge(df, funding_rates_0800, on=['Time'])
df = pd.merge(df, funding_rates_1600, on=['Time'])
df = pd.merge(df, google, on=['Time'])
df = pd.merge(df, active_addresses, on=['Time'])
df = pd.merge(df, unique_addresses, on=['Time'])
df = pd.merge(df, transactions, on=['Time'])
df = pd.merge(tweets, df, how="outer")
df.head()

,Time,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,Close,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,2020-03-12,5.0,2.0,0.0772,0.091,0.100,0.809,7933.2,4802.9,7960.4,4448.6,55248355.79,0.02%,0.01%,0.01%,22.83,6.20,909601,653156.0,349978.0
1,2020-03-17,98.0,96.0,0.4019,0.000,0.114,0.886,5028.8,5316.7,5532.0,4926.1,18269210.08,-0.09%,-0.02%,0.00%,10.21,3.39,765477,540902.0,278326.0
2,2020-03-20,125.0,95.0,0.4767,0.000,0.146,0.854,6162.2,6204.4,6901.7,5661.9,37752569.82,-0.03%,-0.02%,0.00%,10.04,3.56,734614,564748.0,284594.0
3,2020-03-30,3.0,0.0,0.0803,0.061,0.071,0.869,5881.3,6394.9,6600.0,5860.0,17904828.40,-0.03%,-0.06%,-0.03%,5.92,3.63,770915,535787.0,276828.0
4,2020-03-31,186.0,142.0,0.6597,0.000,0.278,0.722,6395.0,6409.7,6520.0,6288.0,13159710.94,-0.01%,-0.01%,-0.02%,5.45,3.54,816889,546702.0,290227.0


In [74]:
print("Type of variables: ", "\n", df.dtypes)

Type of variables:  
 Time                        object
Favorites                  float64
Retweets                   float64
Compound                   float64
Negative                   float64
Positive                   float64
Neutral                    float64
Open                       float64
Close                      float64
High                       float64
Low                        float64
Volume                     float64
Funding Rate 0000           object
Funding Rate 0800           object
Funding Rate 1600           object
Bitcoin_SVI                float64
Cryptocurrency_SVI         float64
No. of Active Addresses      int64
No. of Unique Addresses    float64
Transactions               float64
dtype: object


# EDA & Preprocessing

In [75]:
# Select features, the target output is'Close'
feature_pool = ['Favorites','Retweets','Compound','Negative','Positive',\
                'Neutral','Open', 'High','Low',\
                'Volume','Funding Rate 0000','Funding Rate 0800','Funding Rate 1600','Bitcoin_SVI',\
                'Cryptocurrency_SVI','No. of Active Addresses','No. of Unique Addresses','Transactions']

output = 'Close'

## Data Type Transformation

In [76]:
# Transform object type of "Funding Rate 0000", "Funding Rate 0800" and " Funding Rate 1600" to numerical type
df['Funding Rate 0000'] = df['Funding Rate 0000'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 0800'] = df['Funding Rate 0800'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))
df['Funding Rate 1600'] = df['Funding Rate 1600'].map(lambda x: float('nan') if pd.isnull(x) else float(x.replace('%','')))

## Train-Test Split

In [77]:
X, y = df.drop(output, axis=1), df[output]

# Split the data: train set: test set using = 70/30. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
print(y_train.sum()/y_train.count(),y_test.sum()/y_test.count())

31785.227005870842 32444.582648401825


## Handle Missing Data

In [78]:
# Count missing value
print(df.isna().sum())

# check the consistency of columns with missing value between train set and the whole set
print(df[feature_pool].isnull().any().tolist()==X_train.isnull().any().tolist())
print(df[output].isnull().any().tolist()==y_train.isnull().any().tolist())

Time                         0
Favorites                  458
Retweets                   458
Compound                   458
Negative                   458
Positive                   458
Neutral                    458
Open                         0
Close                        0
High                         0
Low                          0
Volume                       0
Funding Rate 0000            0
Funding Rate 0800            0
Funding Rate 1600            0
Bitcoin_SVI                  0
Cryptocurrency_SVI           0
No. of Active Addresses      0
No. of Unique Addresses      2
Transactions                 0
dtype: int64
False
True


### Monthly Sentiment Analysis

In [79]:
tweets.insert(1, "Month", "NaN")

tweets['Month'] = tweets['Time'].str[:7]
tweets.head(10)

,Time,Month,Favorites,Retweets,Compound,Negative,Positive,Neutral
0,2020-03-12,2020-03,5.0,2.0,0.0772,0.091,0.100,0.809
1,2020-03-17,2020-03,98.0,96.0,0.4019,0.000,0.114,0.886
2,2020-03-20,2020-03,125.0,95.0,0.4767,0.000,0.146,0.854
3,2020-03-30,2020-03,3.0,0.0,0.0803,0.061,0.071,0.869
4,2020-03-31,2020-03,186.0,142.0,0.6597,0.000,0.278,0.722
5,2020-04-01,2020-04,171.0,120.0,0.5267,0.061,0.184,0.754
6,2020-04-16,2020-04,419.0,94.0,0.0000,0.000,0.000,1.000
7,2020-04-22,2020-04,73.0,18.0,0.3182,0.000,0.084,0.916
8,2020-04-30,2020-04,994.0,153.0,0.0000,0.000,0.000,1.000
9,2020-05-09,2020-05,35.0,2.0,0.0000,0.000,0.000,1.000


In [80]:
tweets = tweets.groupby(by="Month").mean()
tweets = tweets.reset_index()

In [81]:
X_train.insert(1, "Month", "NaN")
X_train['Month'] = X_train['Time'].str[:7]

X_test.insert(1, "Month", "NaN")
X_test['Month'] = X_test['Time'].str[:7]

In [82]:
df_X_train = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_train.iterrows():
    if (pd.isna(X_train.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_train.loc[i].at["Month"]]
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_train.loc[i].at["Open"], \
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_train.loc[i].at["Time"], 'Month': X_train.loc[i].at["Month"], 'Favorites': X_train.loc[i].at["Favorites"], 'Retweets': X_train.loc[i].at["Retweets"], 'Compound': X_train.loc[i].at["Compound"], \
                   'Negative': X_train.loc[i].at["Negative"], 'Positive': X_train.loc[i].at["Positive"], 'Neutral': X_train.loc[i].at["Neutral"], 'Open': X_train.loc[i].at["Open"],\
                   'High': X_train.loc[i].at["High"], 'Low': X_train.loc[i].at["Low"], 'Volume': X_train.loc[i].at["Volume"], 'Funding Rate 0000': X_train.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_train.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_train.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_train.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_train.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_train.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_train.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_train.loc[i].at["Transactions"]}
        df_X_train = df_X_train.append(new_row, ignore_index=True)


In [83]:
df_X_test = pd.DataFrame(columns = ['Time', 'Month', 'Favorites', 'Retweets', 'Compound', \
                                 'Negative', 'Positive', 'Neutral', 'Open', 'High', \
                                 'Low', 'Volume', 'Funding Rate 0000', 'Funding Rate 0800', \
                                 'Funding Rate 1600', 'Bitcoin_SVI', 'Cryptocurrency_SVI',  'No. of Active Addresses', \
                                 'No. of Unique Addresses', 'Transactions'])

for i, row in X_test.iterrows():
    if (pd.isna(X_test.loc[i].at["Favorites"])):
        data = tweets[tweets['Month'] == X_test.loc[i].at["Month"]]
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': data.Favorites.values[0], 'Retweets': data.Retweets.values[0], 'Compound': data.Compound.values[0], \
                   'Negative': data.Negative.values[0], 'Positive': data.Positive.values[0], 'Neutral': data.Neutral.values[0], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"], 'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)
    else:
        new_row = {'Time': X_test.loc[i].at["Time"], 'Month': X_test.loc[i].at["Month"], 'Favorites': X_test.loc[i].at["Favorites"], 'Retweets': X_test.loc[i].at["Retweets"], 'Compound': X_test.loc[i].at["Compound"], \
                   'Negative': X_test.loc[i].at["Negative"], 'Positive': X_test.loc[i].at["Positive"], 'Neutral': X_test.loc[i].at["Neutral"], 'Open': X_test.loc[i].at["Open"], \
                   'High': X_test.loc[i].at["High"], 'Low': X_test.loc[i].at["Low"], 'Volume': X_test.loc[i].at["Volume"], 'Funding Rate 0000': X_test.loc[i].at["Funding Rate 0000"], \
                   'Funding Rate 0800': X_test.loc[i].at["Funding Rate 0800"], 'Funding Rate 1600': X_test.loc[i].at["Funding Rate 1600"], 'Bitcoin_SVI': X_test.loc[i].at["Bitcoin_SVI"], \
                   'Cryptocurrency_SVI': X_test.loc[i].at["Cryptocurrency_SVI"], 'No. of Active Addresses': X_test.loc[i].at["No. of Active Addresses"], 'No. of Unique Addresses': X_test.loc[i].at["No. of Unique Addresses"], \
                   'Transactions': X_test.loc[i].at["Transactions"]}
        df_X_test = df_X_test.append(new_row, ignore_index=True)


In [84]:
X_test = df_X_test
X_test = X_test.drop(['Time'], axis=1)
X_test = X_test.drop(['Month'], axis=1)
X_test

,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,2113.750000,253.750000,0.293231,0.026500,0.124750,0.848688,31520.2,32447.0,31112.0,1.055483e+08,0.00,0.01,0.01,12.60,16.73,651828,422214.0,161122.0
1,3980.000000,1006.000000,0.421500,0.000000,0.098000,0.902000,11764.4,11906.7,11325.0,6.010891e+07,0.04,0.03,0.02,5.29,4.92,1049182,707225.0,326499.0
2,1199.634921,155.000000,0.385409,0.007032,0.134857,0.858127,53558.5,57936.4,53042.6,2.775319e+08,0.01,0.01,0.01,14.58,31.96,1122022,738101.0,289389.0
3,1103.925926,241.740741,0.379407,0.016296,0.146444,0.837407,11753.4,11879.7,11667.4,3.592290e+07,0.02,0.02,0.01,6.48,6.03,994559,705709.0,333683.0
4,74.400000,21.400000,0.070720,0.063800,0.078800,0.857400,33100.7,34693.8,32300.0,1.226633e+08,0.04,0.01,0.04,15.27,19.88,999221,724841.0,280863.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,300.000000,50.000000,0.405267,0.000000,0.146000,0.854000,13557.3,14100.0,13412.8,3.520058e+07,-0.01,-0.02,-0.02,7.43,4.43,875335,613906.0,275796.0
215,254.666667,37.833333,0.117625,0.049250,0.123000,0.827750,13550.8,14060.6,13277.5,3.586149e+07,0.01,-0.02,0.00,6.65,4.24,1029970,727413.0,331673.0
216,267.000000,15.000000,0.000000,0.000000,0.000000,1.000000,37252.7,38837.7,35210.0,3.809772e+08,0.01,-0.02,0.01,51.00,47.00,918673,603588.0,223800.0
217,232.000000,155.000000,0.401900,0.000000,0.072000,0.928000,9464.6,9484.6,9278.4,1.432157e+07,0.01,-0.01,-0.01,5.24,3.72,854866,582343.0,283897.0


In [85]:
X_train = df_X_train
X_train = X_train.drop(['Time'], axis=1)
X_train = X_train.drop(['Month'], axis=1)
X_train

,Favorites,Retweets,Compound,Negative,Positive,Neutral,Open,High,Low,Volume,Funding Rate 0000,Funding Rate 0800,Funding Rate 1600,Bitcoin_SVI,Cryptocurrency_SVI,No. of Active Addresses,No. of Unique Addresses,Transactions
0,1870.122024,278.058036,0.376858,0.009379,0.122107,0.868536,44850.4,45984.0,44728.0,1.848977e+08,0.01,0.01,0.01,14.05,22.77,823454,552535.0,218445.0
1,1639.000000,193.000000,0.000000,0.000000,0.000000,1.000000,8838.0,9224.3,8815.5,2.136135e+07,0.00,0.00,0.01,5.92,4.00,913075,603823.0,292927.0
2,1341.000000,316.000000,0.476700,0.000000,0.147000,0.853000,56880.3,58951.8,54147.5,3.281356e+08,0.04,0.04,0.02,15.36,19.65,1110541,808119.0,296119.0
3,1506.138889,242.305556,0.242513,0.019000,0.102708,0.878375,48440.6,52623.9,48100.0,1.608016e+08,0.01,0.01,0.01,16.34,15.52,1185450,840642.0,333402.0
4,1273.000000,71.500000,0.101150,0.000000,0.065000,0.935000,32253.6,32911.0,30824.1,1.208482e+08,0.04,0.01,0.01,16.96,16.73,1182517,768537.0,336935.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,283.285714,49.142857,0.002414,0.027857,0.057429,0.914571,9021.2,9397.8,8911.6,3.772273e+07,0.01,0.01,0.01,8.82,4.77,972369,671519.0,333395.0
507,414.250000,96.250000,0.211225,0.015250,0.067000,0.917500,6794.6,7040.0,6604.5,2.602287e+07,0.01,0.01,0.00,5.76,3.63,750332,548300.0,278763.0
508,3000.500000,419.000000,0.246950,0.000000,0.099000,0.901000,47624.7,48001.0,45425.6,4.305229e+08,0.01,0.01,0.01,12.52,29.74,979005,660755.0,263495.0
509,2113.750000,253.750000,0.293231,0.026500,0.124750,0.848688,32872.8,34100.0,32265.0,1.125778e+08,-0.03,-0.01,-0.02,9.05,13.23,818441,569423.0,227511.0


### Unique Addresses

In [86]:
KNN_miss_filling = KNNImputer(n_neighbors=2).fit(X_train)
X_train = pd.DataFrame(KNN_miss_filling.transform(X_train), columns=feature_pool)
X_test = pd.DataFrame(KNN_miss_filling.transform(X_test), columns=feature_pool)

In [87]:
print(X_train.isna().sum())
print(X_test.isna().sum())

Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
No. of Unique Addresses    0
Transactions               0
dtype: int64
Favorites                  0
Retweets                   0
Compound                   0
Negative                   0
Positive                   0
Neutral                    0
Open                       0
High                       0
Low                        0
Volume                     0
Funding Rate 0000          0
Funding Rate 0800          0
Funding Rate 1600          0
Bitcoin_SVI                0
Cryptocurrency_SVI         0
No. of Active Addresses    0
N

## Preprocessing

In [88]:
# Standardized features:
standardized_features = ['Favorites','Retweets','Compound','Negative','Positive','Neutral','Bitcoin_SVI']     
non_standardized_features = list(set(feature_pool)-set(standardized_features))
    
# Get the scaler based on train set
scaler = preprocessing.StandardScaler().fit(X_train[standardized_features])
print('The mean and standard deviation of training set:', scaler.mean_, scaler.scale_)

# Standardize the features of train and test sets by the scaler of training set
X_train_std=pd.DataFrame(scaler.transform(X_train[standardized_features]))  # transform() return 'numpy.ndarray', not 'DataFrame' or 'Series'
X_train_nstd=pd.DataFrame(X_train[non_standardized_features])

X_test_std=pd.DataFrame(scaler.transform(X_test[standardized_features]))  # return 'numpy.ndarray', not 'DataFrame' or 'Series'
X_test_nstd=pd.DataFrame(X_test[non_standardized_features])

# Rename the columns of X_train_std and X_test_std
X_train_std.columns = X_train_std.columns.map(lambda x: standardized_features[x])
X_test_std.columns = X_test_std.columns.map(lambda x: standardized_features[x])

# Re-combine the normalized and non-normalized data together for both train ans test sets (3 pts)
X_train_std.reset_index(drop=True, inplace=True)
X_train_nstd.reset_index(drop=True, inplace=True)
X_train = pd.concat([X_train_std,X_train_nstd], sort=False,axis=1)
X_test_std.reset_index(drop=True, inplace=True)
X_test_nstd.reset_index(drop=True, inplace=True)
X_test = pd.concat([X_test_std,X_test_nstd], sort=False,axis=1)

# Update the order within feature pool
feature_pool = list(X_train.columns)

The mean and standard deviation of training set: [1.20181637e+03 1.95388828e+02 2.63314912e-01 2.47614501e-02
 1.12946436e-01 8.62291010e-01 1.37324853e+01] [1.69779658e+03 3.11364724e+02 2.21666109e-01 3.74777191e-02
 6.25449463e-02 7.18609853e-02 9.66066667e+00]


In [89]:
X_test.to_csv('../data/X_test.csv', index=False)

In [90]:
X_train.to_csv('../data/X_train.csv', index=False)

In [91]:
y_train.to_csv('../data/Y_train.csv', index=False)

In [92]:
y_test.to_csv('../data/Y_test.csv', index=False)